In [1]:
import os
import time

import torch.nn.functional as F
import torch
from torch import nn
from torchvision import models
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

#Edit root_dir here to point where the TextureImagesDataset folder 
root_dir = '/content/drive/My Drive/Colab Notebooks/A8'

class TextureImages(object):
    def __init__(self, subset='train', batch_size=64, shuffle=True):
        if subset == 'train':
            images = np.load(os.path.join(root_dir, 'TextureImagesDataset',
                                          'train_images.npy'))
            masks = np.load(os.path.join(root_dir, 'TextureImagesDataset',
                                         'train_masks.npy'))
        elif subset == 'test':
            images = np.load(os.path.join(root_dir, 'TextureImagesDataset',
                                          'test_images.npy'))
            masks = np.load(os.path.join(root_dir, 'TextureImagesDataset',
                                         'test_masks.npy'))
        else:
            raise NotImplementedError
        self._images = images
        self.images = self._images
        self._masks = masks
        self.masks = self._masks
        self.batch_size = batch_size
        self.num_samples = len(self.images)
        self.shuffle = shuffle
        if self.shuffle:
            self.shuffle_samples()
        self.next_batch_pointer = 0

    def shuffle_samples(self):
        image_indices = np.random.permutation(np.arange(self.num_samples))
        self.images = self._images[image_indices]
        self.masks = self._masks[image_indices]

    def get_next_batch(self):
        num_samples_left = self.num_samples - self.next_batch_pointer
        if num_samples_left >= self.batch_size:
            x_batch = self.images[self.next_batch_pointer:self.next_batch_pointer + self.batch_size]
            y_batch = self.masks[self.next_batch_pointer:self.next_batch_pointer + self.batch_size]
            self.next_batch_pointer += self.batch_size
        else:
            x_partial_batch_1 = self.images[self.next_batch_pointer:self.num_samples]
            y_partial_batch_1 = self.masks[self.next_batch_pointer:self.num_samples]
            if self.shuffle:
                self.shuffle_samples()
            x_partial_batch_2 = self.images[0:self.batch_size - num_samples_left]
            y_partial_batch_2 = self.masks[0:self.batch_size - num_samples_left]
            x_batch = np.vstack((x_partial_batch_1, x_partial_batch_2))
            y_batch = np.vstack((y_partial_batch_1, y_partial_batch_2))
            self.next_batch_pointer = self.batch_size - num_samples_left
        return x_batch, y_batch

class CrossEntropyLoss2d(nn.Module):
    def __init__(self, weight=None, size_average=True, ignore_index=255):
        super(CrossEntropyLoss2d, self).__init__()
        self.nll_loss = nn.NLLLoss(weight, size_average, ignore_index)

    def forward(self, inputs, targets):
        return self.nll_loss(F.log_softmax(inputs, dim=1), targets)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
class SemSeg(nn.Module):
    # TODO: Implement Semantic Segmentation network here
    # Returned logits must be a tensor of size:
    # (<batch_size>, image_height, image_width, num_classes + 1)
    # 1st dimension is batch dimension
    # image_height and image_width are the height and width of input_tensor
    # last dimension is the softmax dimension. There are 4 texture classes plus 1 background class
    # therefore last dimension will be 5
    def __init__(self):
      super(SemSeg, self).__init__()
      # encoder convolutional layers
      self.encoder1 = nn.Conv2d(3, 12, kernel_size=5)
      self.encoder2 = nn.Conv2d(12, 36, kernel_size=5)
      self.encoder3 = nn.Conv2d(36, 52, kernel_size=5)

      # decoder convolutional layers
      self.decoder1 = nn.ConvTranspose2d(in_channels=52, out_channels=36, kernel_size=5, stride=2,output_padding=1)
      self.decoder2 = nn.ConvTranspose2d(in_channels=36, out_channels=12, kernel_size=5, stride=2,output_padding=1)
      self.output = nn.ConvTranspose2d(in_channels=12, out_channels=5, kernel_size=5, stride=2,output_padding=1)

      # Xavier initialization
      torch.nn.init.xavier_normal_(self.encoder1.weight)
      torch.nn.init.xavier_normal_(self.encoder2.weight)
      torch.nn.init.xavier_normal_(self.encoder3.weight)
      torch.nn.init.xavier_normal_(self.decoder1.weight)
      torch.nn.init.xavier_normal_(self.decoder2.weight)
      torch.nn.init.xavier_normal_(self.output.weight)

    def forward(self, x):
      x = F.relu(self.encoder1(x))
      x1 = F.max_pool2d(x, kernel_size=2, stride=2)
      x2 = F.relu(self.encoder2(x1))
      x3 = F.max_pool2d(x2, kernel_size=2, stride=2)
      x4 = F.relu(self.encoder3(x3))
      x5 = F.max_pool2d(x4, kernel_size=2, stride=2)
      x6 = F.relu(self.decoder1(x5))
      x7 = F.relu(self.decoder2(x6))
      x8 = self.output(x7)
      return F.log_softmax(x8, dim=1)


In [3]:
def run():
    # You can tune the hyperparameters here.
    n_epochs = 25
    batch_size = 16
    learning_rate = 0.001
    weight_decay = 0.001
    use_cuda = 1

    load_weights = 0
    wts_fname = 'model.pt'

    input_size = (196, 196)
    n_batches = int(2000 / batch_size)
    wts_path = os.path.join(root_dir, wts_fname)

    if use_cuda and torch.cuda.is_available():
        device = torch.device("cuda")
        print('Training on GPU: {}'.format(torch.cuda.get_device_name(0)))
    else:
        device = torch.device("cpu")
        print('Training on CPU')

    train_set = TextureImages('train', batch_size=batch_size)
    test_set = TextureImages('test', shuffle=False)

    model = SemSeg().to(device)

    def evaluation(images, true_labels):
        eval_batch_size = 100
        predicted_labels = []
        model.eval()
        with torch.no_grad():
            for start_index in range(0, len(images), eval_batch_size):
                end_index = start_index + eval_batch_size
                batch_x = images[start_index: end_index]
                # batch_x = np.reshape(batch_x, (batch_x.shape[0], 3, 196, 196))
                batch_x = torch.FloatTensor(batch_x).permute((0, 3, 1, 2)).to(device)
                batch_predicted_logits = model(batch_x)
                batch_predicted_labels = torch.argmax(batch_predicted_logits, axis=1)
                batch_predicted_labels = batch_predicted_labels.cpu().numpy()
                predicted_labels += list(batch_predicted_labels)
        predicted_labels = np.vstack(predicted_labels).flatten()
        true_labels = true_labels.flatten()
        accuracy = float((predicted_labels == true_labels).astype(np.int32).sum()) / true_labels.size
        return predicted_labels, accuracy

    if load_weights:      
        print('Loading weights from: {}'.format(wts_path))
        chkpt = torch.load(wts_path, map_location=device)  # load checkpoint
        model.load_state_dict(chkpt['model'])
    else:
        criterion = CrossEntropyLoss2d().to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

        print("Training...")
        mean_loss = 0
        steps = 0
        losses = []
        max_accuracy = 0
        max_accuracy_id = 0
        for epch_id in range(n_epochs):
            model.train()
            for batch_id in range(n_batches):
                start_t = time.time()

                batch_x, batch_y = train_set.get_next_batch()

                batch_x = torch.FloatTensor(batch_x).permute((0, 3, 1, 2)).to(device)
                batch_y = torch.LongTensor(batch_y).squeeze().to(device)

                optimizer.zero_grad()

                outputs = model(batch_x)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()

                end_t = time.time()

                _loss = loss.item()
                steps += 1
                mean_loss += (_loss - mean_loss) / steps
                losses.append(_loss)

                time_taken = end_t - start_t

                #print('batch: {} / {} loss: {} mean_loss: {} time_taken: {}'.format(
                #    batch_id, n_batches, _loss, mean_loss, time_taken))

            _, test_accuracy = evaluation(test_set._images, test_set._masks)
            if test_accuracy > max_accuracy:
                max_accuracy = test_accuracy
                max_accuracy_id = epch_id
                chkpt = {
                    'model': model.state_dict(),
                }
                torch.save(chkpt, '{}.{}'.format(wts_path, max_accuracy_id))
            print("epch {} / {}: Test Pixel Accuracy = {:.3f} max_accuracy = {:.3f} in epoch {}".format(
                epch_id + 1, n_epochs, test_accuracy, max_accuracy, max_accuracy_id + 1))
        print("Done training. Weights saved to: {}".format(wts_fname))
        chkpt = {
            'model': model.state_dict(),
        }
        torch.save(chkpt, wts_path)

    print('Evaluating on test set')
    _, test_accuracy = evaluation(test_set._images, test_set._masks)
    print("Test Pixel Accuracy = {:.3f}".format(test_accuracy))
    return test_accuracy


if __name__ == '__main__':
    run()


Training on GPU: Tesla K80


/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


Training...
epch 1 / 25: Test Pixel Accuracy = 0.753 max_accuracy = 0.753 in epoch 1
epch 2 / 25: Test Pixel Accuracy = 0.942 max_accuracy = 0.942 in epoch 2
epch 3 / 25: Test Pixel Accuracy = 0.975 max_accuracy = 0.975 in epoch 3
epch 4 / 25: Test Pixel Accuracy = 0.980 max_accuracy = 0.980 in epoch 4
epch 5 / 25: Test Pixel Accuracy = 0.983 max_accuracy = 0.983 in epoch 5
epch 6 / 25: Test Pixel Accuracy = 0.985 max_accuracy = 0.985 in epoch 6
epch 7 / 25: Test Pixel Accuracy = 0.984 max_accuracy = 0.985 in epoch 6
epch 8 / 25: Test Pixel Accuracy = 0.986 max_accuracy = 0.986 in epoch 8
epch 9 / 25: Test Pixel Accuracy = 0.987 max_accuracy = 0.987 in epoch 9
epch 10 / 25: Test Pixel Accuracy = 0.987 max_accuracy = 0.987 in epoch 10
epch 11 / 25: Test Pixel Accuracy = 0.988 max_accuracy = 0.988 in epoch 11
epch 12 / 25: Test Pixel Accuracy = 0.987 max_accuracy = 0.988 in epoch 11
epch 13 / 25: Test Pixel Accuracy = 0.988 max_accuracy = 0.988 in epoch 13
epch 14 / 25: Test Pixel Accura

#Possible Output 
Training on GPU: Tesla K80

Training...

epch 1 / 25: Test Pixel Accuracy = 0.865 max_accuracy = 0.865 in epoch 1

epch 2 / 25: Test Pixel Accuracy = 0.945 max_accuracy = 0.945 in epoch 2

epch 3 / 25: Test Pixel Accuracy = 0.938 max_accuracy = 0.945 in epoch 2

epch 4 / 25: Test Pixel Accuracy = 0.975 max_accuracy = 0.975 in epoch 4

epch 5 / 25: Test Pixel Accuracy = 0.981 max_accuracy = 0.981 in epoch 5

epch 6 / 25: Test Pixel Accuracy = 0.982 max_accuracy = 0.982 in epoch 6

epch 7 / 25: Test Pixel Accuracy = 0.583 max_accuracy = 0.982 in epoch 6

epch 8 / 25: Test Pixel Accuracy = 0.959 max_accuracy = 0.982 in epoch 6

epch 9 / 25: Test Pixel Accuracy = 0.762 max_accuracy = 0.982 in epoch 6

epch 10 / 25: Test Pixel Accuracy = 0.864 max_accuracy = 0.982 in epoch 6

epch 11 / 25: Test Pixel Accuracy = 0.941 max_accuracy = 0.982 in epoch 6

epch 12 / 25: Test Pixel Accuracy = 0.963 max_accuracy = 0.982 in epoch 6

epch 13 / 25: Test Pixel Accuracy = 0.954 max_accuracy = 0.982 in epoch 6

epch 14 / 25: Test Pixel Accuracy = 0.821 max_accuracy = 0.982 in epoch 6

epch 15 / 25: Test Pixel Accuracy = 0.846 max_accuracy = 0.982 in epoch 6

epch 16 / 25: Test Pixel Accuracy = 0.967 max_accuracy = 0.982 in epoch 6

epch 17 / 25: Test Pixel Accuracy = 0.945 max_accuracy = 0.982 in epoch 6

epch 18 / 25: Test Pixel Accuracy = 0.971 max_accuracy = 0.982 in epoch 6

epch 19 / 25: Test Pixel Accuracy = 0.985 max_accuracy = 0.985 in epoch 19

epch 20 / 25: Test Pixel Accuracy = 0.980 max_accuracy = 0.985 in epoch 19

epch 21 / 25: Test Pixel Accuracy = 0.986 max_accuracy = 0.986 in epoch 21

epch 22 / 25: Test Pixel Accuracy = 0.988 max_accuracy = 0.988 in epoch 22

epch 23 / 25: Test Pixel Accuracy = 0.989 max_accuracy = 0.989 in epoch 23

epch 24 / 25: Test Pixel Accuracy = 0.982 max_accuracy = 0.989 in epoch 23

epch 25 / 25: Test Pixel Accuracy = 0.987 max_accuracy = 0.989 in epoch 23

Done training. Weights saved to: model.pt

Evaluating on test set

Test Pixel Accuracy = 0.987